## Python code

### Imports

In [7]:
import os
import sys
import time
import serial
import numpy as np

# Adding the src folder in the current directory as it contains the script for Thymio and local occupancy
sys.path.insert(0, os.path.join(os.getcwd(), 'src'))

from Thymio import Thymio

In [8]:
robot = Thymio.serial(port='COM3', refreshing_rate=0.1)

### Accessible functions for the robot

In [9]:
#dir(robot)

### Read/Write variables

In [4]:
#variables = robot.variable_description()
#for var in variables :
#    print(var)

{'name': '_id', 'offset': 0, 'size': 1}
{'name': 'event.source', 'offset': 1, 'size': 1}
{'name': 'event.args', 'offset': 2, 'size': 32}
{'name': '_fwversion', 'offset': 34, 'size': 2}
{'name': '_productId', 'offset': 36, 'size': 1}
{'name': 'buttons._raw', 'offset': 37, 'size': 5}
{'name': 'button.backward', 'offset': 42, 'size': 1}
{'name': 'button.left', 'offset': 43, 'size': 1}
{'name': 'button.center', 'offset': 44, 'size': 1}
{'name': 'button.forward', 'offset': 45, 'size': 1}
{'name': 'button.right', 'offset': 46, 'size': 1}
{'name': 'buttons._mean', 'offset': 47, 'size': 5}
{'name': 'buttons._noise', 'offset': 52, 'size': 5}
{'name': 'prox.horizontal', 'offset': 57, 'size': 7}
{'name': 'prox.comm.rx._payloads', 'offset': 64, 'size': 7}
{'name': 'prox.comm.rx._intensities', 'offset': 71, 'size': 7}
{'name': 'prox.comm.rx', 'offset': 78, 'size': 1}
{'name': 'prox.comm.tx', 'offset': 79, 'size': 1}
{'name': 'prox.ground.ambiant', 'offset': 80, 'size': 2}
{'name': 'prox.ground.refl

### Variables globales

In [10]:
PROXIMITY_THRESHOLD = 3800 #around 2cm
TIME_SLEEP_OBSTACLE = 0.1 #in seconds
TIME_SLEEP_DATA = 0.5 # 0.2 = 5Hz frequency
MOTOR_SPEED = 80 #both motor are set with this value (one in each direction)

SENSOR_0 = 0
SENSOR_1 = 1
SENSOR_2 = 2
SENSOR_3 = 3
SENSOR_4 = 4
SENSOR_5 = 5
SENSOR_6 = 6
ALL_FRONT_SENSOR = 5
PARTIALY_ALL_FRONT_SENSOR = 4
follow_obstacle = 0
COMPTEUR = 6
iteration = 0

### Main function 

In [17]:
for i in range(1000000): #simulation of big while loop
    print (follow_obstacle)
    
    if follow_obstacle == 1 :
        robot.set_var("motor.left.target", MOTOR_SPEED)        #TO REMOVE
        robot.set_var("motor.right.target", MOTOR_SPEED)       #TO REMOVE
    else :
        
        robot.set_var("motor.left.target", 2**16 - MOTOR_SPEED)        #TO REMOVE
        robot.set_var("motor.right.target", MOTOR_SPEED)               #TO REMOVE
    
    sensor_measurment = robot["prox.horizontal"]
    print(sensor_measurment)
    
    if np.all(np.less(sensor_measurment, PROXIMITY_THRESHOLD)):
        pass
    else:
        obstacle_direction = np.nonzero(sensor_measurment)[0]
        #print(obstacle_direction)
        while((np.any(np.greater(sensor_measurment, PROXIMITY_THRESHOLD)))and(np.all(np.not_equal(obstacle_direction, SENSOR_5)))and(np.all(np.not_equal(obstacle_direction, SENSOR_6)))):
            #robot.set_var('leds.top', 123)
            if(np.shape(obstacle_direction)[0] == ALL_FRONT_SENSOR)or(np.shape(obstacle_direction)[0] == PARTIALY_ALL_FRONT_SENSOR):
                if(sensor_measurment[SENSOR_1]>sensor_measurment[SENSOR_3]):
                    robot.set_var("motor.left.target", MOTOR_SPEED)
                    robot.set_var("motor.right.target", 2**16-MOTOR_SPEED)
                else:
                    robot.set_var("motor.left.target", 2**16-MOTOR_SPEED)
                    robot.set_var("motor.right.target", MOTOR_SPEED)
            else:
                if (np.any(np.equal(obstacle_direction,SENSOR_0)))or(np.any(np.equal(obstacle_direction,SENSOR_1))):
                    robot.set_var("motor.left.target", MOTOR_SPEED)
                    robot.set_var("motor.right.target", 2**16-MOTOR_SPEED)
                elif (np.any(np.equal(obstacle_direction,SENSOR_3)))or(np.any(np.equal(obstacle_direction,SENSOR_4))):
                    robot.set_var("motor.left.target", 2**16-MOTOR_SPEED)
                    robot.set_var("motor.right.target", MOTOR_SPEED)
                elif (np.any(np.equal(obstacle_direction,SENSOR_2))):
                    robot.set_var("motor.left.target", 2**16-MOTOR_SPEED)
                    robot.set_var("motor.right.target", MOTOR_SPEED)
        
            time.sleep(TIME_SLEEP_OBSTACLE)
            
            sensor_measurment = robot["prox.horizontal"]
            obstacle_direction = np.nonzero(sensor_measurment)[0]
            
    if (sensor_measurment[0] > 1000) or (sensor_measurment[4] > 1000) :
        follow_obstacle = 1
        iteration = 0
    else : 
        iteration = iteration + 1
        if iteration == COMPTEUR :
            follow_obstacle = 0
        
    time.sleep(TIME_SLEEP_DATA)

1
[3081, 0, 0, 0, 0, 0, 0]
1
[2934, 0, 0, 0, 0, 0, 0]
1
[1533, 0, 0, 0, 0, 0, 0]
1
[0, 0, 0, 0, 0, 0, 0]
1
[0, 0, 0, 0, 0, 0, 0]
1
[0, 0, 0, 0, 0, 0, 0]
1
[0, 0, 0, 0, 0, 0, 0]
1
[0, 0, 0, 0, 0, 0, 0]
1
[0, 0, 0, 0, 0, 0, 0]
0
[0, 0, 0, 0, 0, 0, 0]
0
[0, 0, 0, 0, 0, 0, 0]
0
[0, 0, 0, 0, 0, 0, 0]
0
[0, 0, 0, 0, 0, 0, 0]
0
[0, 0, 0, 0, 0, 0, 0]
0
[2827, 0, 0, 0, 0, 0, 0]
1
[4659, 0, 0, 0, 0, 0, 0]
1
[0, 0, 0, 0, 0, 0, 0]
1
[0, 0, 0, 0, 0, 0, 0]
1
[0, 0, 0, 0, 0, 0, 0]
1
[0, 0, 0, 0, 0, 0, 0]
1
[0, 0, 0, 0, 0, 0, 0]
1
[0, 0, 0, 0, 0, 0, 0]
0
[0, 0, 0, 0, 0, 0, 0]
0
[0, 0, 0, 0, 0, 0, 0]
0
[0, 0, 0, 0, 0, 0, 0]
0
[0, 0, 0, 0, 0, 0, 0]
0
[0, 0, 0, 0, 0, 0, 0]
0
[4577, 0, 0, 0, 0, 0, 0]
1
[0, 0, 0, 0, 0, 0, 0]
1
[0, 0, 0, 0, 0, 0, 0]
1
[0, 0, 0, 0, 0, 0, 0]
1
[0, 0, 0, 0, 0, 0, 0]
1
[0, 0, 0, 0, 0, 0, 0]
1
[0, 0, 0, 0, 0, 0, 0]
0
[0, 0, 0, 0, 0, 0, 0]
0
[0, 0, 0, 0, 0, 0, 0]
0
[0, 0, 0, 0, 0, 0, 0]
0
[0, 0, 0, 0, 0, 0, 0]
0
[0, 0, 0, 0, 0, 0, 0]
0
[0, 0, 0, 0, 0, 0, 0]
0
[0, 0, 0, 0, 0, 0, 0

KeyboardInterrupt: 

### Tests

ANN

In [ ]:
WEIGHT_L=[40,25,-20,-25,-40, 0,0] #sensor_5 = 30 ; sensor_6 = -10 (here = 0 beacause of the cable)
WEIGHT_R=[-40,-25,-20,25,40, 0,0] #sensor_5 = -10 ; sensor_6 = 30 (here = 0 beacause of the cable)

SENSOR_SCALE = 4000 #used for rescaling the data send by the sensor to give it to the motors
THRESHOLD = 20
MOTOR_SPEED = 80 #both motor are set with this value (one in each direction)

for i in range(10000000): #simulation of big while loop 
    
    robot.set_var("motor.left.target", int(2**16) - MOTOR_SPEED)        #TO REMOVE
    robot.set_var("motor.right.target", MOTOR_SPEED)       #TO REMOVE
    
    sensor_measurment = robot["prox.horizontal"]
    #print(sensor_measurment)
    
    measurement_rescaled = np.divide(sensor_measurment, SENSOR_SCALE)
    #print(measurement_rescaled)
    weighted_measurment_l = np.multiply(measurement_rescaled, WEIGHT_L)
    #print(weighted_measurment_l)
    weighted_measurment_r = np.multiply(measurement_rescaled, WEIGHT_R)
    #print(weighted_measurment_r)

    motor_speed_l = np.sum(weighted_measurment_l)
    motor_speed_r = np.sum(weighted_measurment_r)

    #print(motor_speed_l)
    #print(motor_speed_r)
    
    if np.any(np.greater(motor_speed_l, THRESHOLD)) or np.any(np.greater(motor_speed_r, THRESHOLD)) or np.any(np.less(motor_speed_l, -THRESHOLD)) or np.any(np.less(motor_speed_r, -THRESHOLD)):       
        
        obstacle_direction = np.nonzero(sensor_measurment)[0]
        
        if (np.any(np.equal(obstacle_direction,SENSOR_2))) and (np.shape(obstacle_direction)[0]==1):
                    robot.set_var("motor.left.target", 2**16-MOTOR_SPEED)
                    robot.set_var("motor.right.target", MOTOR_SPEED)
        else :
            if motor_speed_l<0:
                #print(2**16 + int(motor_speed_l))
                robot.set_var("motor.left.target", int(2**16) + int(motor_speed_l)-1)
            else :
                robot.set_var("motor.left.target", int(motor_speed_l))

            if motor_speed_r<0:
                #print(2**16 + int(motor_speed_r))
                robot.set_var("motor.right.target", int(2**16) + int(motor_speed_r)-1)
            else :
                robot.set_var("motor.right.target", int(motor_speed_r))
        
        
        """
        robot.set_var("motor.left.target", 0)
        robot.set_var("motor.right.target", 0)
        
        robot.set_var("motor.left.target", 2**16-motor_speed_l)
        robot.set_var("motor.right.target", 2**16-motor_speed_r)
        """
    
    time.sleep(TIME_SLEEP_OBSTACLE)

    

If/elif

In [ ]:
for i in range(1000000): #simulation of big while loop
    robot.set_var("motor.left.target", MOTOR_SPEED)        #TO REMOVE
    robot.set_var("motor.right.target", MOTOR_SPEED)       #TO REMOVE
    
    sensor_measurment = robot["prox.horizontal"]
    print(sensor_measurment)                               #TO REMOVE
    
    if np.all(np.less(sensor_measurment, PROXIMITY_THRESHOLD)):
        pass
        #break ?
    else:
        obstacle_direction = np.nonzero(sensor_measurment)[0]
        
        while((np.any(np.greater(sensor_measurment, PROXIMITY_THRESHOLD)))and(np.all(np.not_equal(obstacle_direction, SENSOR_5)))and(np.all(np.not_equal(obstacle_direction, SENSOR_6)))):
            if(np.shape(obstacle_direction)[0] == ALL_FRONT_SENSOR)or(np.shape(obstacle_direction)[0] == PARTIALY_ALL_FRONT_SENSOR):
                if(sensor_measurment[SENSOR_1]>sensor_measurment[SENSOR_3]):
                    robot.set_var("motor.left.target", MOTOR_SPEED)
                    robot.set_var("motor.right.target", 2**16-MOTOR_SPEED)
                else:
                    robot.set_var("motor.left.target", 2**16-MOTOR_SPEED)
                    robot.set_var("motor.right.target", MOTOR_SPEED)
            
            else:
                if (np.any(np.equal(obstacle_direction,SENSOR_0)))or(np.any(np.equal(obstacle_direction,SENSOR_1))):
                    robot.set_var("motor.left.target", MOTOR_SPEED)
                    robot.set_var("motor.right.target", 2**16-MOTOR_SPEED)
                elif (np.any(np.equal(obstacle_direction,SENSOR_3)))or(np.any(np.equal(obstacle_direction,SENSOR_4))):
                    robot.set_var("motor.left.target", 2**16-MOTOR_SPEED)
                    robot.set_var("motor.right.target", MOTOR_SPEED)
                elif (np.any(np.equal(obstacle_direction,SENSOR_2))):
                    robot.set_var("motor.left.target", 2**16-MOTOR_SPEED)
                    robot.set_var("motor.right.target", MOTOR_SPEED)
        
            print(sensor_measurment)                      #TO REMOVE
            time.sleep(TIME_SLEEP_OBSTACLE)
            
            sensor_measurment = robot["prox.horizontal"]
            obstacle_direction = np.nonzero(sensor_measurment)[0]
        
    time.sleep(TIME_SLEEP_DATA)
robot.set_var("motor.left.target", MOTOR_SPEED)   #TO REMOVE
robot.set_var("motor.right.target", MOTOR_SPEED)  #TO REMOVE

In [18]:
sensor_measurment = robot["prox.horizontal"]
obstacle_direction = np.nonzero(sensor_measurment)[0]

print(sensor_measurment)
print(obstacle_direction)
#print(np.argmax(sensor_measurment))
print((np.shape(obstacle_direction)[0] == 1) and (np.any(np.equal(obstacle_direction,SENSOR_0)) or np.any(np.equal(obstacle_direction,SENSOR_4))))
print(np.any(np.greater(sensor_measurment, PROXIMITY_THRESHOLD)))
print(np.all(np.not_equal(obstacle_direction, SENSOR_5)))
print(np.all(np.not_equal(obstacle_direction, SENSOR_6)))
print(np.all(np.less(sensor_measurment, PROXIMITY_THRESHOLD)))
print(np.any(np.equal(obstacle_direction,SENSOR_2)))

#robot.set_var("motor.left.target", 50)
#robot.set_var("motor.right.target", 2**16-50)
#time.sleep(1.1)
robot.set_var("motor.left.target", 0)
robot.set_var("motor.right.target", 0)

[0, 0, 0, 0, 0, 0, 0]
[]
False
False
True
True
True
False


### Bugs to fix

1) if the robot meet a v shape obstacle  
2) if the robot has an obstacle in front of sensor n°2 it can turns and then sense the obstacle on the 3th, it turns backwards --> wait a bit more in the case of detection on the sensor 2 ?